# Potential

In [4]:
%load_ext memory_profiler
%load_ext snakeviz
%load_ext cython
%load_ext autoreload
%autoreload 2


from IPython.core import debugger
ist = debugger.set_trace

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler
The snakeviz extension is already loaded. To reload it, use:
  %reload_ext snakeviz
The cython extension is already loaded. To reload it, use:
  %reload_ext cython
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
from py.typyMagics import *
ipy = get_ipython()
ipy.register_magics(typyMagics)

In [6]:
import sys
sys.path.insert(0,'../')

## BASE POTENTIAL

In [36]:
%%run_and_write ../typyPRISM/potential/Potential.py
import numpy as np
class Potential:
    '''Baseclass for all intermolecular-pairwise potentials
    
    .. warning:: 
    
     Currently, this class doesn't do anything besides group all of the
     potentials under a single inheritance heirarchy. This will likely
     change as needs arise.
    
    All potentials should inherit from Potential and these subclasses should
    all implement a calculate method. All potential parameters should be specified
    in the constructor except for real-space gridpoints to evaluate the 
    potential at. This information will be provided via  Domain object 
    specified in the System.
    
    '''
        

Overwriting ../typyPRISM/potential/Potential.py


## LENNARD JONES

In [39]:
%%run_and_write ../typyPRISM/potential/LennardJones.py
from typyPRISM.potential.Potential import Potential
import numpy as np
class LennardJones(Potential):
    '''12-6 Lennard-Jones potential
    
    .. math::
    
        U(r) = 4 * \epsilon * ((\sigma/r)^(12.0) - (\sigma/r)^(6.0))
    
    
    Parameters
    ----------
    epsilon: float
        Depth of attractive well
        
    sigma: float
        Contact distance (i.e. low distance where potential magnitude = 0)
    
    rcut: float, *optional*
        Cutoff distance for potential. Useful for comparing directly to results
        from simulations where cutoffs are necessary. 
    
    shift: bool,*optional*
        Shift the potential by its value at the cutoff. Clearly this only makes 
        sense if rcut is specified
    
    '''
    def __init__(self,epsilon,sigma,rcut=None,shift=False):
        self.epsilon = epsilon
        self.sigma = sigma
        self.rcut  = rcut
        self.shift = shift
        self.funk  = lambda r: 4 * epsilon * ((sigma/r)**(12.0) - (sigma/r)**(6.0))
        
    def __repr__(self):
        return '<Potential: LennardJones>'
        
    def calculate(self,r):
        magnitude = self.funk(r)
        
        if self.rcut is not None:
            if self.shift:
                magnitude -= self.funk(self.rcut)
            magnitude[r>self.rcut] = 0.0
                
        return magnitude
        

Overwriting ../typyPRISM/potential/LennardJones.py


In [17]:
%%run_and_write ../typyPRISM/test/LennardJones_TestCase.py
import unittest
from typyPRISM.potential.LennardJones import LennardJones
import numpy as np

class LennardJones_TestCase(unittest.TestCase):
    def test_create(self):
        '''Can we create a LennardJones potential?'''
        r = np.arange(0.75,3.5,0.05)
        epsilon = 0.25
        sigma   = 1.05
        rcut    = 2.5*sigma
        shift   = True
        
        UShift = 4*epsilon*((sigma/rcut)**(12.0) - (sigma/rcut)**(6.0))
        U1 = 4*epsilon*((sigma/r)**(12.0) - (sigma/r)**(6.0)) - UShift
        U1[r>rcut] = 0
        
        U2 = LennardJones(epsilon,sigma,rcut=rcut,shift=shift).calculate(r)
        np.testing.assert_array_almost_equal(U1,U2)
        
        


Overwriting ../typyPRISM/test/LennardJones_TestCase.py


## Weeks-Chandler-Andersen

In [38]:
%%run_and_write ../typyPRISM/potential/WeeksChandlerAndersen.py
from typyPRISM.potential.LennardJones import LennardJones
import numpy as np
class WeeksChandlerAndersen(LennardJones):
    '''Purely repulsive Weeks-Chandler-Andersen potential
    
    .. math::
    
        U(r) = 4 * \epsilon * ((\sigma/r)^(12.0) - (\sigma/r)^(6.0)) + epsilon, r<rcut
        U(r) = 0.0, r>=rcut
        r_{cut} = 2^(1.0/6.0) * \sigma
    
    
    Parameters
    ----------
    epsilon: float
        Repulsive strength modifier
        
    sigma: float
        Contact distance 
    
    '''
    def __init__(self,epsilon,sigma):
        rcut = sigma * 2**(1.0/6.0)
        super().__init__(epsilon,sigma,rcut=rcut,shift=True)
        
    def __repr__(self):
        return '<Potential: WeeksChandlerAndersen>'
        
        

Overwriting ../typyPRISM/potential/WeeksChandlerAndersen.py


In [33]:
%%run_and_write ../typyPRISM/test/WeeksChandlerAndersen_TestCase.py
import unittest
from typyPRISM.potential.WeeksChandlerAndersen import WeeksChandlerAndersen
import numpy as np

class WeeksChandlerAndersen_TestCase(unittest.TestCase):
    def test_create(self):
        '''Can we create a WeeksChandlerAndersen potential?'''
        r = np.arange(0.75,3.5,0.05)
        epsilon = 0.25
        sigma   = 1.05
        rcut    = 2.0**(1.0/6.0)*sigma
        shift   = True
        
        UShift = 4*epsilon*((sigma/rcut)**(12.0) - (sigma/rcut)**(6.0))
        U1 = 4*epsilon*((sigma/r)**(12.0) - (sigma/r)**(6.0)) - UShift
        U1[r>rcut] = 0
        
        U2 = WeeksChandlerAndersen(epsilon,sigma).calculate(r)
        np.testing.assert_array_almost_equal(U1,U2)
        
        


Overwriting ../typyPRISM/test/WeeksChandlerAndersen_TestCase.py


## HARD SPHERE

In [37]:
%%run_and_write ../typyPRISM/potential/HardSphere.py
from typyPRISM.potential.Potential import Potential
import numpy as np
class HardSphere(Potential):
    '''Simple hard sphere potential
    
    .. math::
    
        U(r>=\sigma)  = 0.0
        U(r<\sigma) = high_value
    
    
    Parameters
    ----------
    sigma: float
        Contact distance 
    
    high_value: float, *optional*
        Cutoff distance for potential. Useful for comparing directly to results
        from simulations where cutoffs are necessary. 
    
    
    '''
    def __init__(self,sigma,high_value=1e6):
        self.sigma = sigma
        self.high_value = high_value
        self.funk  = lambda r: np.where(r>=sigma,0.0,high_value)
    def __repr__(self):
        return '<Potential: HardSphere>'
    
    def calculate(self,r):
        magnitude = self.funk(r)
        return magnitude
        

Overwriting ../typyPRISM/potential/HardSphere.py


In [26]:
%%run_and_write ../typyPRISM/test/HardSphere_TestCase.py
import unittest
from typyPRISM.potential.HardSphere import HardSphere
import numpy as np

class HardSphere_TestCase(unittest.TestCase):
    def test_create(self):
        '''Can we create a HardSphere potential?'''
        r = np.arange(0.75,3.5,0.05)
        sigma   = 1.05
        high_value = 1e5
        
        U1 = np.zeros_like(r)
        U1[r<sigma] = high_value
        
        U2 = HardSphere(sigma,high_value).calculate(r)
        np.testing.assert_array_almost_equal(U1,U2)
        
        


Overwriting ../typyPRISM/test/HardSphere_TestCase.py


## RUN ALL

In [34]:
import unittest
suite = []
suite.append(unittest.TestLoader().loadTestsFromTestCase(LennardJones_TestCase))
suite.append(unittest.TestLoader().loadTestsFromTestCase(HardSphere_TestCase))
suite.append(unittest.TestLoader().loadTestsFromTestCase(WeeksChandlerAndersen_TestCase))
suite = unittest.TestSuite(suite)
unittest.TextTestRunner(verbosity=2).run(suite)

test_create (__main__.LennardJones_TestCase)
Can we create a LennardJones potential? ... ok
test_create (__main__.HardSphere_TestCase)
Can we create a LennardJones potential? ... ok
test_create (__main__.WeeksChandlerAndersen_TestCase)
Can we create a WeeksChandlerAndersen potential? ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.007s

OK


<unittest.runner.TextTestResult run=3 errors=0 failures=0>